# Data Conversion of HydroCAT .hex data
This python notebook shows how to convert a .hex file into pandas dataframe with scientific values.

Please contact [SBS customer support](https://www.seabird.com/support) for help or to request additional features.

## Init

The initialization code section below is used to import required libraries.

In [ ]:
# Native imports
import os

# Third-party imports
import numpy as np
import pandas as pd
import gsw

# Sea-Bird imports
import seabirdscientific.conversion as conv
import seabirdscientific.instrument_data as id
import seabirdscientific.visualization as viz
import example_data.example_coefficients as ec

## [Data Conversion](#proc-list)

This section shows how to convert raw data contained in a .hex file into scientific units for the instruments that follow:  
- HydroCAT
- HydroCAT-ODO

In [ ]:
hex_file = os.path.join("example_data", "HydroCAT", "HydroCAT-ODO.hex")

# Convert raw hexadecimal string to raw frequencies
raw_data = id.read_hex_file(
    filepath=hex_file,
    instrument_type=id.InstrumentType.HydroCATODO,
    enabled_sensors=[
        id.Sensors.Temperature,
        id.Sensors.Conductivity,
        id.Sensors.Pressure,
        id.Sensors.SBE63
    ],
)

In [ ]:
# Convert raw frequencies to scientific values

temperature = conv.convert_temperature(
    temperature_counts_in=raw_data["temperature"].values,
    coefs=ec.temperature_coefs_sn03906502,
    standard='ITS90',
    units='C',
    use_mv_r=False,
)

pressure_dbar = conv.convert_pressure(
    pressure_count=raw_data["pressure"].values,
    compensation_voltage=raw_data["temperature compensation"],
    coefs=ec.pressure_coefs_sn03906502,
    units='dbar',
) 

conductivity = conv.convert_conductivity(
    conductivity_count=raw_data["conductivity"].values,
    temperature=temperature,
    pressure=pressure_dbar,  # must be in units of dbar
    coefs=ec.conductivity_coefs_sn6385
)

# salinity = gsw.SP_from_C(
#     C=conductivity / 1000, # mS/cm
#     t=temperature, # ITS90 C
#     p=pressure_dbar # dbar
# )
salinity = np.full(len(temperature), 30.0) # override until we have a real data set

oxygen = conv.convert_sbe63_oxygen(
    raw_oxygen_phase=raw_data["SBE63 oxygen phase"],
    thermistor=raw_data["SBE63 oxygen temperature"],
    pressure=pressure_dbar,
    salinity=salinity,
    coefs=ec.oxygen_63_coefs_sn2568,
    thermistor_coefs=ec.thermistor_63_coefs_sn2568,
    thermistor_units="volts"
)

# Flag to be used in data processing
flag = np.zeros(len(temperature))

data = pd.DataFrame({
    "temp": temperature,
    "cond":conductivity,
    "sal":salinity,
    "oxy":oxygen,
    "press":pressure_dbar,
    "date_time":raw_data["date time"],
    "flag":flag
})

data

## [Data Plotting](#proc-list)

In [ ]:
config = viz.ChartConfig(
    title="HydroCAT-ODO Data Conversion",
    x_names=["date_time"],
    y_names=["temp", "cond", "press", "oxy"],
    z_names=[],
    chart_type="overlay",
    plot_loop_edit_flags=False,
    lift_pen_over_bad_data=True,
)

chart_data = viz.ChartData(data, config)
fig = viz.plot_xy_chart(chart_data, config)

# plotly customizations
fig.data[0].name = "Temperature"
fig.data[1].name = "Conductivity"
fig.data[2].name = "Pressure"
fig.data[3].name = "Oxygen"
fig["layout"]["yaxis"]["title"] = "Temperature [ITS-90 degrees C]"
fig["layout"]["yaxis2"]["title"] = "Conductivity [S/m]"
fig["layout"]["yaxis3"]["title"] = "Pressure [dbar]"
fig["layout"]["yaxis4"]["title"] = "Oxygen [ml/l]"


fig.update_layout(height=800)
fig.show()